## Pthreads API
* Q3: _What are all possible outputs produced by this program?_
    ```C
    1   #include <pthread.h>
    2   void *printer(void *arg) {
    3       char *ch = (char*)arg;
    4       printf("I am %c\n", *ch);
    5       return NULL;
    6   }
    7   int main() {
    8       pthread_t thread1, thread2;
    9       char *ch1 = malloc(sizeof(char));
    10      *ch1 = 'X';
    11      char *ch2 = malloc(sizeof(char));
    12      *ch2 = 'Y';
    13      pthread_create(&thread1, NULL, &printer, ch1);
    14      pthread_create(&thread2, NULL, &printer, ch2);
    15      pthread_join(thread1, NULL);
    16      pthread_join(thread2, NULL);
    17  }
    ```
    ```
    I am X	
    I am Y
    ```
    OR
    ```
	I am Y
	I am X
    ```

## Transforming programs to be multi-threaded

* For-each style loops are natural candidates for threading
    * Example: a program that counts the number of occurrences of each vowel within a string
    * Another example: a program that counts the longest sequence of each Short Tandem Repeat (STR) in a DNA sequence — you already performed loop optimization on such a program last week; this week in lab you'll make it multi-threaded
* Steps
    * Move the body of the loop into its own "worker" function (if not done already)
    * Write a "wrapper" function which:
        * Has the required signature for a thread function (i.e., takes a single `void *` parameter and returns a void pointer)
        * Extracts the argument(s) for the "worker" function from the thread function's arguments
        * Calls the "worker" function
        * Prepares the return value from the "worker" function to be returned by the thread
    * Replace the body of the loop with the creation of threads that execute the "wrapper" function
    * Add a second loop to wait for all threads to complete

## Warm-up
Q1: _Consider the following program:_
```C
1   #include <ctype.h>
2   #include <pthread.h>
3   #include <stdio.h>
4   #include <stdlib.h>
5   #include <string.h>
6   int count_upper(char *str) {
7       int count = 0;
8       for (int i = 0; i < strlen(str); i++) {
9          if (isupper(str[i])) {
10              count++;
11          }
12      }
13      return count;
14  }
15  int main(int argc, char *argv[]) {
16      if (argc < 2) {
17          printf("Error: provide a string\n");
18          return 1;
19      }
20      char *str = argv[1];
21      pthread_t thr;
22      pthread_create(thr, NULL, &count_upper, str);
23      int count = 0;
24      pthread_join(thr, &count);
25      printf("There are %d uppercase letters\n", count);
26  }
```
_Compiling this program with `clang` results in the following warnings:_
```
buggy.c:22:20: warning: incompatible integer to pointer conversion passing 
'pthread_t' (aka 'unsigned long') to parameter of type 'pthread_t *' (aka 
'unsigned long *'); take the address with & [-Wint-conversion]
    pthread_create(thr, NULL, &count_upper, str);
                   ^~~
                   &
/usr/include/pthread.h:198:50: note: passing argument to parameter 
'__newthread' here
extern int pthread_create (pthread_t *__restrict __newthread,
                                                 ^

buggy.c:22:31: warning: incompatible function pointer types passing 
'int (*)(char *)' to parameter of type 'void *(*)(void *)' 
[-Wincompatible-function-pointer-types]
    pthread_create(thr, NULL, &count_upper, str);
                              ^~~~~~~~~~~~
/usr/include/pthread.h:200:15: note: passing argument to parameter 
'__start_routine' here
                           void *(*__start_routine) (void *),
                                   ^

buggy.c:24:23: warning: incompatible pointer types passing 
'int *' to parameter of type 'void **' [-Wincompatible-pointer-types]
    pthread_join(thr, &count);
                      ^~~~~~
/usr/include/pthread.h:215:49: note: passing argument to parameter 
'__thread_return' here
extern int pthread_join (pthread_t __th, void **__thread_return);
                                                ^
3 warnings generated.
```
_How would you change the code to fix these problems?_
* Need to pass `&thr` to `pthread_create` (instead of `thr`) on line 22
* Function executed by a thread must return `void *` and take a single `void *` parameter; replace lines 6-7 with:
    ```C
    void *count_upper(void *arg) {
        char *str = (char *)arg;
        int *count = malloc(sizeof(int));
        *count = 0;
    ```
    Also replace line 10 with:
    ```C
    *count++;
    ```
* Need to pass a double pointer to `pthread_join` on line 24; replace lines 23-25 with:
    ```C
    int *count = NULL;
    pthread_join(thr, &count);
    printf("There are %d uppercase letters\n", *count);
    ``` 

## Making programs multi-threaded
* _Assume you are given the following code:_
    ```C
    #include <ctype.h>
    #include <stdio.h>
    #include <string.h>
    int count(char *str, char ch) {
        int num = 0;
        while (*str != '\0') {
            if (tolower(*str) == ch) {
                num++;
            }
            str++;
        }
        return num;
    }
    int main(int argc, char *argv[]) {
        if (argc < 2) {
            printf("Error: provide a string\n");
            return 1;
        }
        char *str = argv[1];
        char *vowels = "aeiou";
        int counts[strlen(vowels)];
        for (int i = 0; i < strlen(vowels); i++) {
            counts[i] = count(str, vowels[i]);
        }

        for (int i = 0; i < strlen(vowels); i++) {
            printf("%c %d\n", vowels[i], counts[i]);
        }
    }
    ```
* Q2: *Write a function called  `count_wrapper` that has the signature required for a thread function and calls the `count` function. (Hint: you'll need to declare a `struct` that contains all of the parameters required for `count`.)*
    ```C
    struct count_args {
        char *str;
        char ch;
    };
    void *count_wrapper(void *arg) {
        struct count_args *cargs = (struct count_args *)arg;
        int *result = malloc(sizeof(int));
        *result = count(cargs->str, cargs->ch);
        return result;
    }
    ```
* Q3: *Re-write `main` to create/wait for threads that execute `count_wrapper` (instead of calling `count` sequentially).*
    ```C
    int main(int argc, char *argv[]) {
        if (argc < 2) {
            printf("Error: provide a string\n");
            return 1;
        }
        char *str = argv[1];
        char *vowels = "aeiou";

        // Create threads
        struct count_args args[strlen(vowels)];
        pthread_t threads[strlen(vowels)];
        for (int i = 0; i < strlen(vowels); i++) {
            args[i].str = str;
            args[i].ch = vowels[i];
            pthread_create(&(threads[i]), NULL, &count_wrapper, &(args[i]));
        }

        // Wait for threads
        for (int i = 0; i < strlen(vowels); i++) {
            int *ptr;
            pthread_join(threads[i], (void **)(&ptr));
            printf("%c %d\n", vowels[i], *ptr);
        }
    }
    ```